In [ ]:
import wikipediaapi

# Overview
1. Gather a list of each company in the S&P 500
   1. From Wikipedia
      1. Company Name
      2. Ticker
      3. Headquarters Location
      4. GCIS Sub-Industry
      5. Year Founded
   2. From 

In [ ]:
# Check if a page exists
wiki_wiki = wikipediaapi.Wikipedia('Joeys company guesser project', 'en')

# Save the contents of the page within a variable
page_py = wiki_wiki.page('List of S&P 500 companies')

# Save the contents of the page as text
page_text = page_py.text

def print_sections(sections, level=0):
    """ Gather all of the top level sections within a particular wikipedia page

    Args:
        sections (str): _description_
        level (int, optional): _description_. Defaults to 0.
    """
    for s in sections:
        # if "S&P 500 component stocks" in s.title:
            # print(s.title)
            # print(s.text[0:200])
        print("%s: %s - %s" % ("*" * (level + 1), s.title, s.text[0:4000]))
        print_sections(s.sections, level + 1)
print_sections(page_py.sections)

In [ ]:
# print("Page - Title: %s" % page_py.title)

# print("Page - Summary: %s" % page_py.summary[0:6090])
# print(page_py.text)

In [ ]:
wiki_wiki = wikipediaapi.Wikipedia(
    user_agent='Joeys company guesser project',
        language='en',
        extract_format=wikipediaapi.ExtractFormat.WIKI
)

# Beautiful Soup Approach

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Get the page's HTML content
URL = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
response = requests.get(URL)

# Convert the table to a Pandas DataFrame
df = pd.read_html(response.content, flavor='lxml')[0]

# Display the first few rows of the table
display(df)

### Find random company

In [ ]:
import random
N = len(df)
for i in range(100):
    pick_random_company = random.randint(0,N-1)
    if pick_random_company > 500:
        print(pick_random_company)
        
print(pick_random_company)
display(df.iloc[pick_random_company])

### Get wiki link for each company & append it to the dataframe in a new column

In [ ]:
response = requests.get(URL)
soup = BeautifulSoup(response.text, 'html.parser')

# Find the table
table = soup.find('table', {'class': 'wikitable'})

first_row = table.find_all('tr')[0]
first_link = first_row.find_all('th')[2]

# # print(first_row)
# count = 0
# for row in table.find_all('tr')[1:2]:
#     print(row)
#     print(count)
#     count += 1

# Find all of the links in the second column of the table
links = []
for row in table.find_all('tr')[1:]: # skip the header row
    cells = row.find_all('td')
    if cells:
        link_tag = cells[1].find('a')  # Adjust the index based on the column containing the link
        if link_tag and 'href' in link_tag.attrs:
            links.append(link_tag['href'])

# Create column in existing dataframe with link as element, but with full url
updated_df = df.assign(link=links)
updated_df['wiki_link'] = 'https://en.wikipedia.org' + updated_df['link']
display(updated_df)

# save the dataframe to a CSV file
updated_df.to_csv('S&P_500_companies.csv', index=False)



In [ ]:


def get_revenue_from_page(sp500_df):

    # loop through the dataframe and get the revenue for each company
    for index, row in sp500_df.iterrows():
        print(index,row)
        # # Get the HTML content of the linked page
        # link = updated_df.iloc[index]['wiki_link']
        # page_response = requests.get(link)
        # page_soup = BeautifulSoup(page_response.text, 'html.parser')

        # # find the table that include information regarding revenue
        # table = page_soup.find_all('table', {'class': 'infobox ib-company vcard'})
        # for row in table:
        #     revenue = row.find_all('tr')[12]
        #     revenue2 = revenue.find_all('span')[2].text
        #     span_with_title = revenue.find('span').find('span', title=True)
        #     # Extract the title attribute
        #     title = span_with_title['title'] if span_with_title else None
        #     print(title, revenue2[2:])

In [ ]:
get_revenue_from_page(updated_df)